# Team Name : HyperTensor

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

## Correlation Matrix

In [128]:
# Loading Data
df = pd.read_csv('Stock_Data_06-11.csv', names = ['0','symbol','last','change','changePercentage','bidSize','bid','offer','offerSize','turnover','high','low','open','lastVolume','totalTrades','lastTrade','timeStamp'])
df.corr()

,0,last,change,changePercentage,bidSize,bid,offer,offerSize,turnover,high,low,open,lastVolume,totalTrades
0,1.000000,-0.040078,0.005349,0.004074,0.032734,-0.009548,-0.018547,-0.013656,-0.008321,-0.038810,-0.039062,-0.038917,-0.021968,0.034415
last,-0.040078,1.000000,-0.007630,-0.012236,-0.036783,0.802529,0.826924,-0.039731,-0.142495,0.941644,0.941716,0.961938,-0.070606,-0.095012
change,0.005349,-0.007630,1.000000,0.466332,0.032411,0.023764,-0.070792,-0.012881,0.013851,-0.015116,-0.015312,-0.026247,0.002622,0.037051
changePercentage,0.004074,-0.012236,0.466332,1.000000,0.138810,0.003663,-0.043139,-0.070401,0.103503,-0.014840,-0.014812,-0.021523,0.013436,0.094208
bidSize,0.032734,-0.036783,0.032411,0.138810,1.000000,-0.029508,-0.043740,-0.001502,0.060239,-0.034318,-0.034268,-0.038930,0.007063,0.053750
bid,-0.009548,0.802529,0.023764,0.003663,-0.029508,1.000000,0.924146,-0.040787,-0.133155,0.771140,0.772472,0.845728,-0.064641,-0.117503
offer,-0.018547,0.826924,-0.070792,-0.043139,-0.043740,0.924146,1.000000,-0.033144,-0.132421,0.792046,0.793497,0.873740,-0.066116,-0.112314
offerSize,-0.013656,-0.039731,-0.012881,-0.070401,-0.001502,-0.040787,-0.033144,1.000000,0.086359,-0.037113,-0.037170,-0.040919,0.008055,0.077287
turnover,-0.008321,-0.142495,0.013851,0.103503,0.060239,-0.133155,-0.132421,0.086359,1.000000,-0.131379,-0.131707,-0.148586,0.062394,0.563775
high,-0.038810,0.941644,-0.015116,-0.014840,-0.034318,0.771140,0.792046,-0.037113,-0.131379,1.000000,0.999851,0.905560,-0.066726,-0.067313


In [129]:
# dropping first column i.e id and some other columns based on correlation
df = df.drop(['0','lastTrade','offer','change','bidSize','offerSize','lastVolume','totalTrades'],axis = 1)
df.head()

,symbol,last,changePercentage,bid,turnover,high,low,open,timeStamp
0,HBL,170.51,-0.40,170.70,58800,171.90,165.00,171.19,2018-06-01 09:00:54
1,AKBL,22.07,0.00,21.52,0,0.00,0.00,22.07,2018-06-01 09:00:54
2,FABL,25.98,0.00,25.60,0,0.00,0.00,25.98,2018-06-01 09:00:54
3,SNGP,96.70,-0.41,96.71,4200,97.01,96.70,97.10,2018-06-01 09:00:54
4,ANL,13.01,-1.14,13.02,1000,13.01,13.01,13.16,2018-06-01 09:00:54


## Steps Performed
### 1. Subset data on every stock
### 2. Prepocessing on that data (Creating/Dropping Column, Filling missing value)
### 3. Normalizing Data
### 4. Fitting the data in LSTM model
### 5. Creation of regression model for each attribute
### 6. Filling the data of attributes for  Future Predictions
### 7. Predicting the future data for specific stock and writng into csv files

In [6]:
companies = df['symbol'].unique()

# Looping through each stock 
for company in companies:
    ## Subset Data based on current company
    my_dat = df[df['symbol'] == company]
    ## Creating new attribute 'avg' based on 'low' and 'high' attribute
    my_dat['avg'] = (my_dat['low'] + my_dat['high'])/2.0
    
    ## Dropping 'low' and 'high' 
    my_dat = my_dat.drop(['high','low'],axis = 1)
    
    ## Setting Timestamp as index
    my_dat.set_index('timeStamp', inplace=True)

    ## Coverting zeros into avg of specific column
    data = np.array(my_dat)
    l = [3,4,5,6]
    for i in l:
        data[:,i][data[:,i] == 0 ] = np.mean(data[:,i])

    # Ceating input_data
    input_feature= my_dat.iloc[:,2:].values
    input_data = input_feature

    ## Normalizing values between 0 and 1
    sc= MinMaxScaler(feature_range=(0,1))
    X = sc.fit_transform(input_feature[:,:])

    # Target variable
    Y = data[:,1]
    Y = Y.astype('float')
    Y = Y.reshape(len(Y),1)

    # Transforming data to fit into LSTM model
    X2 = X.reshape(X.shape[0],1, X.shape[1])

    # Creating LSTM Model
    model = Sequential()
    model.add(LSTM(units=30, return_sequences= True, input_shape=(X2.shape[1],X2.shape[2])))
    model.add(LSTM(units=30, return_sequences=True))
    model.add(LSTM(units=30))
    model.add(Dense(units=1))

    model.summary()

    # Compile Model
    model.compile(optimizer='adam', loss='mean_squared_error')
    history = model.fit(X2, Y, epochs=25,batch_size=32, verbose=1)

    ## 2nd Part
    ## Creating column containing values from 1 to size of data
    one = np.arange(X.shape[0] + 1)
    one = one[1:]
    one = one.reshape(X.shape[0],1)
    
    ## Concatenating the column with input data
    X = np.concatenate((one,X),axis=1)

    # reading sample_solution and dropping 'id' and 'last_price'
    test_dat = pd.read_csv('sample_solution.csv',)
    test_dat = test_dat.drop(['id','last_price'],axis= 1)

    
    my_dat2 = test_dat.loc[test_dat['symbol'] == company]
    X_test = np.arange(X.shape[0],X.shape[0]+my_dat2.shape[0]+1,1)

    X_test = X_test[1:]
    X_test = X_test.reshape(X_test.shape[0],1)

    att_array = np.zeros((my_dat2.shape[0],5))

    # Fitting regression model and generating data of features
    atts = [1,2,3,4,5]
    poly_features = PolynomialFeatures(degree=2, include_bias=True)
    x_poly = poly_features.fit_transform(X[:,0].reshape(X.shape[0],1))
    Xtest_poly = poly_features.fit_transform(X_test[:,0].reshape(X_test.shape[0],1))
    for i in atts:
        linearModel = LinearRegression()
        linearModel.fit(x_poly, X[:,i].reshape(X.shape[0],1))
        y_poly_pred = linearModel.predict(Xtest_poly)
        #att_array[:,i] = att_array[:,i].reshape(att_array.shape(0),1)
        y_poly_pred = y_poly_pred.reshape(-1)
        att_array[:,i-1] = y_poly_pred
    
    # Prediction using our trained models
    att_array = att_array.reshape(att_array.shape[0],1,att_array.shape[1])
    pred = model.predict(att_array)
    my_dat2 = np.array(my_dat2)

    sol = np.concatenate((my_dat2,pred),axis = 1)
    
    sol = pd.DataFrame(sol)
    with open('wahab.csv','a') as fd:
        sol.to_csv(fd)
    


        


/home/wahab/anaconda3/envs/tensorflowenv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 1, 30)             4320      
_________________________________________________________________
lstm_11 (LSTM)               (None, 1, 30)             7320      
_________________________________________________________________
lstm_12 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 31        
Total params: 18,991
Trainable params: 18,991
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
54282/54282 [==============================] - 17s 305us/step - loss: 16357.1300
Epoch 2/25
54282/54282 [==============================] - 14s 261us/step - loss: 8280.2283
Epoch 3/25
54282/54282 [==============================] - 14s 253us/step - loss: 3558.1

/home/wahab/anaconda3/envs/tensorflowenv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 1, 30)             4320      
_________________________________________________________________
lstm_14 (LSTM)               (None, 1, 30)             7320      
_________________________________________________________________
lstm_15 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 18,991
Trainable params: 18,991
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
54280/54280 [==============================] - 16s 286us/step - loss: 54.9140
Epoch 2/25
54280/54280 [==============================] - 12s 220us/step - loss: 1.3205
Epoch 3/25
54280/54280 [==============================] - 12s 221us/step - loss: 1.3083
Epoch

/home/wahab/anaconda3/envs/tensorflowenv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


...

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_136 (LSTM)              (None, 1, 30)             4320      
_________________________________________________________________
lstm_137 (LSTM)              (None, 1, 30)             7320      
_________________________________________________________________
lstm_138 (LSTM)              (None, 30)                7320      
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 31        
Total params: 18,991
Trainable params: 18,991
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
54276/54276 [==============================] - 27s 504us/step - loss: 32499.7713
Epoch 2/25
54276/54276 [==============================] - 16s 293us/step - loss: 20212.4004
Epoch 3/25
54276/54276 [==============================] - 18s 326us/step - loss: 11698

/home/wahab/anaconda3/envs/tensorflowenv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_139 (LSTM)              (None, 1, 30)             4320      
_________________________________________________________________
lstm_140 (LSTM)              (None, 1, 30)             7320      
_________________________________________________________________
lstm_141 (LSTM)              (None, 30)                7320      
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 31        
Total params: 18,991
Trainable params: 18,991
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
54276/54276 [==============================] - 28s 522us/step - loss: 3.4146
Epoch 2/25
54276/54276 [==============================] - 15s 273us/step - loss: 0.7838
Epoch 3/25
54276/54276 [==============================] - 15s 272us/step - loss: 0.6758
Epoch 

/home/wahab/anaconda3/envs/tensorflowenv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_142 (LSTM)              (None, 1, 30)             4320      
_________________________________________________________________
lstm_143 (LSTM)              (None, 1, 30)             7320      
_________________________________________________________________
lstm_144 (LSTM)              (None, 30)                7320      
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 31        
Total params: 18,991
Trainable params: 18,991
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
54276/54276 [==============================] - 29s 528us/step - loss: 45.8792
Epoch 2/25
54276/54276 [==============================] - 17s 314us/step - loss: 5.3548
Epoch 3/25
54276/54276 [==============================] - 19s 358us/step - loss: 5.1383
Epoch

## Converting generated csv file to acceptable format for submission on Kaggle

In [110]:
# Processing data to convert to acceptable submission format

sf = pd.read_csv('wahab2.csv',names= ['id','symbol','timestamp','last'], header = None)
sf['symbol'] = sf['symbol'].replace('0',np.nan)
sf = sf.dropna()
sf['timestamp'] =pd.to_datetime(sf.timestamp)
sf = sf.sort_values(by='timestamp',ascending = True)
timestamps = sf['timestamp'].unique()

ids = np.array(sf['id'])
ids= ids.reshape(ids.shape[0],1)

symbols = np.array(sf['symbol'])
symbols= symbols.reshape(symbols.shape[0],1)

lasts = np.array(sf['last'])
lasts= lasts.reshape(lasts.shape[0],1)

timestamps = np.array(sf['timestamp'])
timestamps= timestamps.reshape(timestamps.shape[0],1)

timestamps = timestamps.astype('str')

sol_array = np.concatenate((ids,symbols,lasts,timestamps),axis = 1)

sol_array = pd.DataFrame(sol_array)
sol_array.columns = ['id','symbol','last_price','timestamp']

#sol_array['timestamp'] = pd.to_datetime(sol_array['timestamp'])
sample = pd.read_csv('sample_solution.csv')
sol_array['id'] = sample['id']

#sol_array.to_csv('finalsolution10.csv', index = False)

sample = np.array(sample)
p = np.zeros((743581,4))
p = p.astype('O')
sol_array = np.array(sol_array)

for company in companies:
    inds = np.where(sample[:,1]==company)
    inds_sol = np.where(sol_array[:,1] == company)
    for i in range(len(inds)):
        p[inds[i],:] = sol_array[inds_sol[i],:]

p = pd.DataFrame(p)
p.columns = ['id','symbol','last_price','timestamp']

#sol_array['timestamp'] = pd.to_datetime(sol_array['timestamp'])
sample = pd.read_csv('sample_solution.csv')
p['id'] = sample['id']
p['timestamp'] = pd.to_datetime(p['timestamp'])
p.to_csv('grandfinal3.csv',index=False)

        

## Reference

https://medium.com/datadriveninvestor/multivariate-time-series-using-rnn-with-keras-7f78f4488679